# Aggregating counts and rates of hospital admissions and make a table

In [14]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

## 0- functions and read data

In [15]:
def data_process(data, data_hosped):
    #replace SMs with 5 in data
    for row in range(0, len(data)):
        if data['count'][row] == 'SM':
            data['count'][row] = 5
    #conver counts into int
    data['count_int'] = data['count'].astype(str).replace('\.0', '', regex=True).astype(int)
    data_hosped['count_int'] = data['count_int']
    #replace SMs with 5 in data_hosped
    for row in range(0, len(data_hosped)):
        if data_hosped['count'][row] == 'SM':
            data_hosped['count'][row] = 5
    data_hosped['count_hosped_int'] = data_hosped['count'].astype(str).replace('\.0', '', regex=True).astype(int)
    data_hosped['hosped_rate'] = (data_hosped['count_hosped_int']/data['count_int'])*1000
    #custom order of variables to fit tables of the manuscript
    data_hosped['variable'] = pd.Categorical(data_hosped['variable'], ["age_cat_15_24", "age_cat_25_34", "age_cat_35_44", "age_cat_45_54", "age_cat_55_64", "age_cat_65_74", "age_cat_75_more", "sex_male", "sex_female", "bmi_cat_underweight", "bmi_cat_healthy_weight", "bmi_cat_overweight", "bmi_cat_obese", "bmi_cat_unknown", "ethnicity_white", "ethnicity_non-white", "ethnicity_unknown", "CCI_cat_very_low", "CCI_cat_low", "CCI_cat_medium", "CCI_cat_high", "CCI_cat_very_high", "smoking_status_smoker", "smoking_status_ex_smoker", "smoking_status_never_smoked", "smoking_status_unknown", "imd_very_affluent", "imd_affluent", "imd_medium", "imd_unaffluent", "imd_very_unaffluent", "imd_unknown", "season_spring", "season_summer", "season_autumn", "season_winter", "region_london", "region_north_east", "region_north_west", "region_east", "region_west_midlands", "region_yorkshire", "region_south_east", "region_east_midlands", "region_south_west", "flu_vaccine_yes", "flu_vaccine_no", "antibacterial_brit", "period_prepandemic", "period_during_pandemic", "period_post_2nd_lockdown"])
    data_hosped = data_hosped.sort_values('variable')
    #keep mean values of gp_count and antibacterial_brit
    data_hosped['hosped_rate'] = np.round(data_hosped['hosped_rate'], decimals=1)
    #reset index to prevent issue in calculating total value
    data_hosped = data_hosped.reset_index()
    # add comma after every 3 digits
    data_hosped['count_hosped_int']=data_hosped['count_hosped_int'].astype(int)
    data_hosped['count_hosped_int'] = data_hosped['count_hosped_int'].apply(lambda x : "{:,}".format(x))
    #combine counts and means
    for row in range(0, len(data_hosped)):
        if data_hosped['count'][row] == 5:
            data_hosped['count_hosped_int'][row] = 'SN'
            data_hosped['hosped_rate'][row] = 'SN'    
    #combine conts and rates
    data_hosped['count_rate_hosped'] = data_hosped['count_hosped_int'].astype(str) + ' (' + data_hosped['hosped_rate'].astype(str) + ')'
    #calculate total from sex_male and sex_female
    data_hosped['count'] = data_hosped['count'].astype(str).replace('\.0', '', regex=True).astype(int)
    total = int(data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_male'].index[0]),int(data_hosped.columns.get_loc('count'))]) + int(data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_female'].index[0]),int(data_hosped.columns.get_loc('count'))])
    total = "{:,}".format(total)
    new_row = {'index':'', 'variable':'Total', 'count':'', 'mean':'', 'std':'', 'count_int':'', 'count_hosped_int':'', 'hosped_rate':'', 'count_rate_hosped':total} #count_hosped_int, hosped_rate, count_rate_hosped
    #append row to the dataframe
    data_hosped = data_hosped.append(new_row, ignore_index=True)
    #keep variable names and counts comibed with means
    data_hosped = data_hosped[['variable', 'count_rate_hosped']]
    #custom order of variables to fit tables of the manuscript again
    data_hosped['variable'] = pd.Categorical(data_hosped['variable'], ["Total", "age_cat_15_24", "age_cat_25_34", "age_cat_35_44", "age_cat_45_54", "age_cat_55_64", "age_cat_65_74", "age_cat_75_more", "sex_male", "sex_female", "bmi_cat_underweight", "bmi_cat_healthy_weight", "bmi_cat_overweight", "bmi_cat_obese", "bmi_cat_unknown", "ethnicity_white", "ethnicity_non-white", "ethnicity_unknown", "CCI_cat_very_low", "CCI_cat_low", "CCI_cat_medium", "CCI_cat_high", "CCI_cat_very_high", "smoking_status_smoker", "smoking_status_ex_smoker", "smoking_status_never_smoked", "smoking_status_unknown", "imd_very_affluent", "imd_affluent", "imd_medium", "imd_unaffluent", "imd_very_unaffluent", "imd_unknown", "season_spring", "season_summer", "season_autumn", "season_winter", "region_london", "region_north_east", "region_north_west", "region_east", "region_west_midlands", "region_yorkshire", "region_south_east", "region_east_midlands", "region_south_west", "flu_vaccine_yes", "flu_vaccine_no", "antibacterial_brit", "period_prepandemic", "period_during_pandemic", "period_post_2nd_lockdown"])
    data_hosped = data_hosped.sort_values('variable')

    return data_hosped

In [16]:
summary_table_hosped_lrti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_incdt_hosped.csv')
summary_table_hosped_lrti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_incdt_hosped.csv')
summary_table_hosped_lrti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_prevl_hosped.csv')
summary_table_hosped_lrti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_prevl_hosped.csv')
summary_table_hosped_urti_combined_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_incdt_hosped.csv')
summary_table_hosped_urti_combined_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_incdt_hosped.csv')
summary_table_hosped_urti_combined_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_prevl_hosped.csv')
summary_table_hosped_urti_combined_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_prevl_hosped.csv')
summary_table_hosped_uti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_incdt_hosped.csv')
summary_table_hosped_uti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_incdt_hosped.csv')
summary_table_hosped_uti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_prevl_hosped.csv')
summary_table_hosped_uti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_prevl_hosped.csv')
summary_table_hosped_sinusitis_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_incdt_hosped.csv')
summary_table_hosped_sinusitis_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_incdt_hosped.csv')
summary_table_hosped_sinusitis_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_prevl_hosped.csv')
summary_table_hosped_sinusitis_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_prevl_hosped.csv')
summary_table_hosped_otmedia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_incdt_hosped.csv')
summary_table_hosped_otmedia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_incdt_hosped.csv')
summary_table_hosped_otmedia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_prevl_hosped.csv')
summary_table_hosped_otmedia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_prevl_hosped.csv')
summary_table_hosped_ot_externa_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_incdt_hosped.csv')
summary_table_hosped_ot_externa_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_incdt_hosped.csv')
summary_table_hosped_ot_externa_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_prevl_hosped.csv')
summary_table_hosped_ot_externa_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_prevl_hosped.csv')
summary_table_hosped_pneumonia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_no_abs_incdt_hosped.csv')
summary_table_hosped_pneumonia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_abs_incdt_hosped.csv')
summary_table_hosped_pneumonia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_no_abs_prevl_hosped.csv')
summary_table_hosped_pneumonia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_abs_prevl_hosped.csv')
summary_table_hosped_urti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_incdt_hosped.csv')
summary_table_hosped_urti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_incdt_hosped.csv')
summary_table_hosped_urti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_prevl_hosped.csv')
summary_table_hosped_urti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_prevl_hosped.csv')
summary_table_hosped_cough_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_incdt_hosped.csv')
summary_table_hosped_cough_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_incdt_hosped.csv')
summary_table_hosped_cough_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_prevl_hosped.csv')
summary_table_hosped_cough_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_prevl_hosped.csv')
summary_table_hosped_cough_cold_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_incdt_hosped.csv')
summary_table_hosped_cough_cold_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_incdt_hosped.csv')
summary_table_hosped_cough_cold_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_prevl_hosped.csv')
summary_table_hosped_cough_cold_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_prevl_hosped.csv')
summary_table_hosped_throat_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_incdt_hosped.csv')
summary_table_hosped_throat_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_incdt_hosped.csv')
summary_table_hosped_throat_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_prevl_hosped.csv')
summary_table_hosped_throat_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_prevl_hosped.csv')

In [17]:
summary_table_lrti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_incdt.csv')
summary_table_lrti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_incdt.csv')
summary_table_lrti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_prevl.csv')
summary_table_lrti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_prevl.csv')
summary_table_urti_combined_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_incdt.csv')
summary_table_urti_combined_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_incdt.csv')
summary_table_urti_combined_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_prevl.csv')
summary_table_urti_combined_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_prevl.csv')
summary_table_uti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_incdt.csv')
summary_table_uti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_incdt.csv')
summary_table_uti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_prevl.csv')
summary_table_uti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_prevl.csv')
summary_table_sinusitis_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_incdt.csv')
summary_table_sinusitis_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_incdt.csv')
summary_table_sinusitis_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_prevl.csv')
summary_table_sinusitis_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_prevl.csv')
summary_table_otmedia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_incdt.csv')
summary_table_otmedia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_incdt.csv')
summary_table_otmedia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_prevl.csv')
summary_table_otmedia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_prevl.csv')
summary_table_ot_externa_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_incdt.csv')
summary_table_ot_externa_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_incdt.csv')
summary_table_ot_externa_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_prevl.csv')
summary_table_ot_externa_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_prevl.csv')
summary_table_pneumonia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_no_abs_incdt.csv')
summary_table_pneumonia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_abs_incdt.csv')
summary_table_pneumonia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_no_abs_prevl.csv')
summary_table_pneumonia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_abs_prevl.csv')
summary_table_urti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_incdt.csv')
summary_table_urti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_incdt.csv')
summary_table_urti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_prevl.csv')
summary_table_urti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_prevl.csv')
summary_table_cough_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_incdt.csv')
summary_table_cough_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_incdt.csv')
summary_table_cough_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_prevl.csv')
summary_table_cough_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_prevl.csv')
summary_table_cough_cold_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_incdt.csv')
summary_table_cough_cold_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_incdt.csv')
summary_table_cough_cold_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_prevl.csv')
summary_table_cough_cold_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_prevl.csv')
summary_table_throat_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_incdt.csv')
summary_table_throat_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_incdt.csv')
summary_table_throat_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_prevl.csv')
summary_table_throat_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_prevl.csv')

## 1- process data to make a table of counts and rates, and save it

In [18]:
summary_table_lrti_no_abs_incdt = data_process(summary_table_lrti_no_abs_incdt, summary_table_hosped_lrti_no_abs_incdt)
summary_table_lrti_abs_incdt = data_process(summary_table_lrti_abs_incdt, summary_table_hosped_lrti_abs_incdt)
summary_table_lrti_no_abs_prevl = data_process(summary_table_lrti_no_abs_prevl, summary_table_hosped_lrti_no_abs_prevl)
summary_table_lrti_abs_prevl = data_process(summary_table_lrti_abs_prevl, summary_table_hosped_lrti_no_abs_prevl)
summary_table_urti_combined_no_abs_incdt = data_process(summary_table_urti_combined_no_abs_incdt, summary_table_hosped_urti_combined_no_abs_incdt)
summary_table_urti_combined_abs_incdt = data_process(summary_table_urti_combined_abs_incdt, summary_table_hosped_urti_combined_abs_incdt)
summary_table_urti_combined_no_abs_prevl = data_process(summary_table_urti_combined_no_abs_prevl, summary_table_hosped_urti_combined_no_abs_prevl)
summary_table_urti_combined_abs_prevl = data_process(summary_table_urti_combined_abs_prevl, summary_table_hosped_urti_combined_abs_prevl)
summary_table_uti_no_abs_incdt = data_process(summary_table_uti_no_abs_incdt, summary_table_hosped_uti_no_abs_incdt)
summary_table_uti_abs_incdt = data_process(summary_table_uti_abs_incdt, summary_table_hosped_uti_abs_incdt)
summary_table_uti_no_abs_prevl = data_process(summary_table_uti_no_abs_prevl, summary_table_hosped_uti_no_abs_prevl)
summary_table_uti_abs_prevl = data_process(summary_table_uti_abs_prevl, summary_table_hosped_uti_abs_prevl)
summary_table_sinusitis_no_abs_incdt = data_process(summary_table_sinusitis_no_abs_incdt, summary_table_hosped_sinusitis_no_abs_incdt)
summary_table_sinusitis_abs_incdt = data_process(summary_table_sinusitis_abs_incdt, summary_table_hosped_sinusitis_abs_incdt)
summary_table_sinusitis_no_abs_prevl = data_process(summary_table_sinusitis_no_abs_prevl, summary_table_hosped_sinusitis_no_abs_prevl)
summary_table_sinusitis_abs_prevl = data_process(summary_table_sinusitis_abs_prevl, summary_table_hosped_sinusitis_abs_prevl)
summary_table_otmedia_no_abs_incdt = data_process(summary_table_otmedia_no_abs_incdt, summary_table_hosped_otmedia_no_abs_incdt)
summary_table_otmedia_abs_incdt = data_process(summary_table_otmedia_abs_incdt, summary_table_hosped_otmedia_abs_incdt)
summary_table_otmedia_no_abs_prevl = data_process(summary_table_otmedia_no_abs_prevl, summary_table_hosped_otmedia_no_abs_prevl)
summary_table_otmedia_abs_prevl = data_process(summary_table_otmedia_abs_prevl, summary_table_hosped_otmedia_abs_prevl)
summary_table_ot_externa_no_abs_incdt = data_process(summary_table_ot_externa_no_abs_incdt, summary_table_hosped_ot_externa_no_abs_incdt)
summary_table_ot_externa_abs_incdt = data_process(summary_table_ot_externa_abs_incdt, summary_table_hosped_ot_externa_abs_incdt)
summary_table_ot_externa_no_abs_prevl = data_process(summary_table_ot_externa_no_abs_prevl, summary_table_hosped_ot_externa_no_abs_prevl)
summary_table_ot_externa_abs_prevl = data_process(summary_table_ot_externa_abs_prevl, summary_table_hosped_ot_externa_abs_prevl)
summary_table_pneumonia_no_abs_incdt = data_process(summary_table_pneumonia_no_abs_incdt, summary_table_hosped_pneumonia_no_abs_incdt)
summary_table_pneumonia_abs_incdt = data_process(summary_table_pneumonia_abs_incdt, summary_table_hosped_pneumonia_abs_incdt)
summary_table_pneumonia_no_abs_prevl = data_process(summary_table_pneumonia_no_abs_prevl, summary_table_hosped_pneumonia_no_abs_prevl)
summary_table_pneumonia_abs_prevl = data_process(summary_table_pneumonia_abs_prevl, summary_table_hosped_pneumonia_abs_prevl)
summary_table_urti_no_abs_incdt = data_process(summary_table_urti_no_abs_incdt, summary_table_hosped_urti_no_abs_incdt)
summary_table_urti_abs_incdt = data_process(summary_table_urti_abs_incdt, summary_table_hosped_urti_abs_incdt)
summary_table_urti_no_abs_prevl = data_process(summary_table_urti_no_abs_prevl, summary_table_hosped_urti_no_abs_prevl)
summary_table_urti_abs_prevl = data_process(summary_table_urti_abs_prevl, summary_table_hosped_urti_abs_prevl)
summary_table_cough_no_abs_incdt = data_process(summary_table_cough_no_abs_incdt, summary_table_hosped_cough_no_abs_incdt)
summary_table_cough_abs_incdt = data_process(summary_table_cough_abs_incdt, summary_table_hosped_cough_abs_incdt)
summary_table_cough_no_abs_prevl = data_process(summary_table_cough_no_abs_prevl, summary_table_hosped_cough_no_abs_prevl)
summary_table_cough_abs_prevl = data_process(summary_table_cough_abs_prevl, summary_table_hosped_cough_abs_prevl)
summary_table_cough_cold_no_abs_incdt = data_process(summary_table_cough_cold_no_abs_incdt, summary_table_hosped_cough_cold_no_abs_incdt)
summary_table_cough_cold_abs_incdt = data_process(summary_table_cough_cold_abs_incdt, summary_table_hosped_cough_cold_abs_incdt)
summary_table_cough_cold_no_abs_prevl = data_process(summary_table_cough_cold_no_abs_prevl, summary_table_hosped_cough_cold_no_abs_prevl)
summary_table_cough_cold_abs_prevl = data_process(summary_table_cough_cold_abs_prevl, summary_table_hosped_cough_cold_abs_prevl)
summary_table_throat_no_abs_incdt = data_process(summary_table_throat_no_abs_incdt, summary_table_hosped_throat_no_abs_incdt)
summary_table_throat_abs_incdt = data_process(summary_table_throat_abs_incdt, summary_table_hosped_throat_abs_incdt)
summary_table_throat_no_abs_prevl = data_process(summary_table_throat_no_abs_prevl, summary_table_hosped_throat_no_abs_prevl)
summary_table_throat_abs_prevl = data_process(summary_table_throat_abs_prevl, summary_table_hosped_throat_abs_prevl)

In [19]:
d = {'variable': summary_table_lrti_no_abs_incdt['variable'],
     
     'lrti_no_abs_incdt': summary_table_lrti_no_abs_incdt['count_rate_hosped'], 
     'lrti_abs_incdt': summary_table_lrti_abs_incdt['count_rate_hosped'],
     'lrti_no_abs_prevl': summary_table_lrti_no_abs_prevl['count_rate_hosped'], 
     'lrti_abs_prevl': summary_table_lrti_abs_prevl['count_rate_hosped'],
     
     'urti_combined_no_abs_incdt': summary_table_urti_combined_no_abs_incdt['count_rate_hosped'], 
     'urti_combined_abs_incdt': summary_table_urti_combined_abs_incdt['count_rate_hosped'],
     'urti_combined_no_abs_prevl': summary_table_urti_combined_no_abs_prevl['count_rate_hosped'], 
     'urti_combined_abs_prevl': summary_table_urti_combined_abs_prevl['count_rate_hosped'],
     
     'uti_no_abs_incdt': summary_table_uti_no_abs_incdt['count_rate_hosped'], 
     'uti_abs_incdt': summary_table_uti_abs_incdt['count_rate_hosped'],
     'uti_no_abs_prevl': summary_table_uti_no_abs_prevl['count_rate_hosped'], 
     'uti_abs_prevl': summary_table_uti_abs_prevl['count_rate_hosped'],
     
     'sinusitis_no_abs_incdt': summary_table_sinusitis_no_abs_incdt['count_rate_hosped'], 
     'sinusitis_abs_incdt': summary_table_sinusitis_abs_incdt['count_rate_hosped'],
     'sinusitis_no_abs_prevl': summary_table_sinusitis_no_abs_prevl['count_rate_hosped'], 
     'sinusitis_abs_prevl': summary_table_sinusitis_abs_prevl['count_rate_hosped'],
     
     'otmedia_no_abs_incdt': summary_table_otmedia_no_abs_incdt['count_rate_hosped'], 
     'otmedia_abs_incdt': summary_table_otmedia_abs_incdt['count_rate_hosped'],
     'otmedia_no_abs_prevl': summary_table_otmedia_no_abs_prevl['count_rate_hosped'], 
     'otmedia_abs_prevl': summary_table_otmedia_abs_prevl['count_rate_hosped'],
     
     'ot_externa_no_abs_incdt': summary_table_ot_externa_no_abs_incdt['count_rate_hosped'], 
     'ot_externa_abs_incdt': summary_table_ot_externa_abs_incdt['count_rate_hosped'],
     'ot_externa_no_abs_prevl': summary_table_ot_externa_no_abs_prevl['count_rate_hosped'], 
     'ot_externa_abs_prevl': summary_table_ot_externa_abs_prevl['count_rate_hosped'],

     'pneumonia_no_abs_incdt': summary_table_pneumonia_no_abs_incdt['count_rate_hosped'], 
     'pneumonia_abs_incdt': summary_table_pneumonia_abs_incdt['count_rate_hosped'],
     'pneumonia_no_abs_prevl': summary_table_pneumonia_no_abs_prevl['count_rate_hosped'], 
     'pneumonia_abs_prevl': summary_table_pneumonia_abs_prevl['count_rate_hosped'],

     'urti_no_abs_incdt': summary_table_urti_no_abs_incdt['count_rate_hosped'], 
     'urti_abs_incdt': summary_table_urti_abs_incdt['count_rate_hosped'],
     'urti_no_abs_prevl': summary_table_urti_no_abs_prevl['count_rate_hosped'], 
     'urti_abs_prevl': summary_table_urti_abs_prevl['count_rate_hosped'],

     'cough_no_abs_incdt': summary_table_cough_no_abs_incdt['count_rate_hosped'], 
     'cough_abs_incdt': summary_table_cough_abs_incdt['count_rate_hosped'],
     'cough_no_abs_prevl': summary_table_cough_no_abs_prevl['count_rate_hosped'], 
     'cough_abs_prevl': summary_table_cough_abs_prevl['count_rate_hosped'],

     'cough_cold_no_abs_incdt': summary_table_cough_cold_no_abs_incdt['count_rate_hosped'], 
     'cough_cold_abs_incdt': summary_table_cough_cold_abs_incdt['count_rate_hosped'],
     'cough_cold_no_abs_prevl': summary_table_cough_cold_no_abs_prevl['count_rate_hosped'], 
     'cough_cold_abs_prevl': summary_table_cough_cold_abs_prevl['count_rate_hosped'],

     'throat_no_abs_incdt': summary_table_throat_no_abs_incdt['count_rate_hosped'], 
     'throat_abs_incdt': summary_table_throat_abs_incdt['count_rate_hosped'],
     'throat_no_abs_prevl': summary_table_throat_no_abs_prevl['count_rate_hosped'], 
     'throat_abs_prevl': summary_table_throat_abs_prevl['count_rate_hosped'],
    }
data = pd.DataFrame(data=d)

data.to_csv(f'../output/aggregate_hosp_table/counts_rates_table_hosped_all_infections.csv', index=False)
data

,variable,lrti_no_abs_incdt,lrti_abs_incdt,lrti_no_abs_prevl,lrti_abs_prevl,urti_combined_no_abs_incdt,urti_combined_abs_incdt,urti_combined_no_abs_prevl,urti_combined_abs_prevl,uti_no_abs_incdt,...,cough_no_abs_prevl,cough_abs_prevl,cough_cold_no_abs_incdt,cough_cold_abs_incdt,cough_cold_no_abs_prevl,cough_cold_abs_prevl,throat_no_abs_incdt,throat_abs_incdt,throat_no_abs_prevl,throat_abs_prevl
0,age_cat_15_24,140 (6.8),10 (4.4),15 (6.6),15 (63.8),600 (7.4),70 (7.8),80 (8.9),10 (9.7),140 (6.9),...,25 (11.2),SN (SN),560 (26.4),60 (25.3),55 (23.2),SN (SN),170 (8.3),20 (8.9),20 (9.1),SN (SN)
1,age_cat_25_34,160 (7.8),20 (8.9),10 (4.4),10 (35.7),595 (7.3),75 (8.2),65 (7.2),10 (9.9),160 (8.0),...,15 (6.6),SN (SN),555 (26.3),75 (30.9),70 (30.2),SN (SN),170 (8.4),20 (8.6),15 (6.7),SN (SN)
2,age_cat_35_44,145 (7.2),20 (9.0),25 (11.0),25 (104.2),575 (7.1),60 (6.6),60 (6.7),10 (9.5),165 (8.2),...,15 (6.9),SN (SN),610 (28.2),75 (31.0),60 (25.1),SN (SN),140 (6.9),15 (6.7),20 (8.8),SN (SN)
3,age_cat_45_54,130 (6.4),20 (9.2),20 (8.8),20 (78.4),640 (7.9),65 (7.0),60 (6.6),SN (SN),155 (7.6),...,15 (6.7),SN (SN),585 (27.2),75 (31.1),70 (29.9),SN (SN),160 (7.9),20 (8.8),20 (8.7),SN (SN)
4,age_cat_55_64,155 (7.7),20 (8.5),10 (4.6),10 (42.6),580 (7.2),60 (6.7),60 (6.8),SN (SN),175 (8.5),...,20 (9.2),SN (SN),625 (29.1),75 (30.9),80 (33.5),10 (34.5),115 (5.6),15 (6.5),15 (6.7),SN (SN)
5,age_cat_65_74,150 (7.3),20 (8.9),20 (9.1),20 (74.1),610 (7.5),55 (6.1),60 (6.5),10 (9.6),155 (7.7),...,20 (8.6),SN (SN),580 (26.8),50 (20.8),80 (32.3),SN (SN),160 (7.9),15 (6.6),15 (6.5),SN (SN)
6,age_cat_75_more,135 (6.6),20 (8.8),15 (6.6),15 (57.7),600 (7.3),70 (7.5),65 (7.1),SN (SN),160 (7.9),...,15 (6.8),SN (SN),610 (28.4),55 (24.1),65 (27.3),SN (SN),145 (7.1),20 (8.8),SN (SN),SN (SN)
7,sex_male,490 (7.0),55 (7.1),60 (7.8),60 (68.2),"2,080 (7.5)",230 (7.4),220 (7.1),25 (7.2),540 (7.8),...,55 (7.3),10 (11.4),"1,985 (27.1)",220 (27.0),255 (31.2),20 (21.9),515 (7.3),65 (8.3),55 (7.1),SN (SN)
8,sex_female,525 (7.2),80 (9.9),60 (7.4),60 (67.0),"2,125 (7.3)",230 (7.1),235 (7.3),30 (8.3),575 (7.9),...,65 (8.1),10 (10.9),"2,145 (27.9)",245 (28.5),225 (26.5),20 (21.4),540 (7.5),65 (8.0),55 (6.8),SN (SN)
9,bmi_cat_underweight,30 (6.6),SN (SN),SN (SN),SN (SN),145 (7.7),20 (9.7),SN (SN),SN (SN),40 (8.3),...,10 (7.6),SN (SN),135 (27.1),10 (18.3),20 (37.4),SN (SN),45 (9.3),SN (SN),10 (7.3),SN (SN)
